# Make your data ready for QGIS

We start with some imports, than you set your project folder and the python script will generate the GeoTiff file you need for QGIS.

## Imports

In [21]:
import os
import shutil
import glob
from osgeo import gdal
import subprocess

## Set your folder

Put your folder location inside of the brackets

In [ ]:
# Set your project folder -  the rest will be done automatically

input_folder = "home/example_folders"

## Transformation from XYZ files to TIFF

Depending on the area you chose this may take a while. 
16km² take around 45s on a Ryzen 3600 with 16GB. 

In [ ]:
# Define input folder and output folder
output_folder = os.path.join(input_folder + "temp")

# Create the output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Walk through all folders and files in the input folder
for root, _, files in os.walk(input_folder):
    for file in files:
        if file.endswith(".xyz"):
            input_path = os.path.join(root, file)
            output_path = os.path.join(output_folder, f"{os.path.splitext(file)[0]}.tif")
            
            # Use gdal_translate to convert .xyz to .tif
            gdal.Translate(
                output_path,
                input_path,
                outputSRS="EPSG:25833",  # Set the projection
                format="GTiff"          # Specify GeoTIFF as output format
            )
            #print(f"Converted: {input_path} -> {output_path}")

print("All .xyz files have been converted into 'geotiffs, now.")
print("Files are beeing transformed")
transformed_files = []

# Loop through all .tif files in the folder
for filename in os.listdir(output_folder):
    if filename.endswith(".tif"):
        # Define the full input and output file paths
        input_file = os.path.join(output_folder, filename)
        output_file = os.path.join(output_folder, f"transformed_{filename}")

        # Define the gdalwarp command with the appropriate EPSG code
        warp_command = [
            "gdalwarp",
            "-t_srs", "EPSG: 25833",
            input_file,
            output_file
        ]
        transformed_files.append(output_file)
        # Run the command
        subprocess.run(warp_command)
        
        print("All files have been transformed to the right projection")

print ("Finished!")
# Remove the geotiff directory


if transformed_files:
    merged_output = os.path.join(input_folder, "merged_transformed.tif")
    
    # Build the gdal_merge command
    merge_command = ["gdal_merge", "-o", merged_output] + transformed_files
    
    # Run the merge command
    subprocess.run(merge_command)
    
    print(f"Merged {len(transformed_files)} files into {merged_output}")
    
else:
    print("No files were transformed, nothing to merge")

shutil.rmtree(output_folder) # Cleanup temporary files


All .xyz files have been converted into 'geotiffs, now.
Files are beeing transformed
Creating output file that is 2000P x 2000L.
Processing dgm1_33436_5640_2_sn.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
All files have been transformed to the right projection
Creating output file that is 2000P x 2000L.
Processing dgm1_33436_5642_2_sn.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
All files have been transformed to the right projection
Creating output file that is 2000P x 2000L.
Processing dgm1_33436_5646_2_sn.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
All files have been transformed to the right projection
Creating output file that is 2000P x 2000L.
Processing dgm1_33436_5644_2_sn.tif [1/1] : 0...10...20...30...40...50...60...70...80...90...100 - done.
All files have been transformed to the right projection
Finished!
0...10...20...30...40...50...60...70...80...90...100 - done.
Merged 4 files into /hom